In [1]:
import os
import shutil
import sys
import re
import json
from google.colab import drive

MY_ZIP_NAME = "VLPromptLearner.zip"

%cd /content
!rm -rf PromptSRC Dassl.pytorch
!pip uninstall -y shap > /dev/null 2>&1
!pip install "numpy<2.0" gdown > /dev/null 2>&1

!git clone https://github.com/muzairkhattak/PromptSRC.git
!git clone https://github.com/KaiyangZhou/Dassl.pytorch.git


%cd /content/Dassl.pytorch
torchtools_path = "dassl/utils/torchtools.py"
with open(torchtools_path, "r") as f:
    code = f.read()
code = code.replace("checkpoint = torch.load(fpath, map_location=map_location)",
                    "checkpoint = torch.load(fpath, map_location=map_location, weights_only=False)")
with open(torchtools_path, "w") as f:
    f.write(code)

!pip install -r requirements.txt > /dev/null 2>&1
!python setup.py develop > /dev/null 2>&1


%cd /content/PromptSRC
!pip install -r requirements.txt > /dev/null 2>&1


DATA_ROOT = "/content/PromptSRC/data"
FLOWERS_DIR = os.path.join(DATA_ROOT, "oxford_flowers")
os.makedirs(FLOWERS_DIR, exist_ok=True)
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"

!curl -L -A "{USER_AGENT}" -o {DATA_ROOT}/102flowers.tgz https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
!curl -L -A "{USER_AGENT}" -o {FLOWERS_DIR}/imagelabels.mat https://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat
!curl -L -A "{USER_AGENT}" -o {FLOWERS_DIR}/setid.mat https://www.robots.ox.ac.uk/~vgg/data/flowers/102/setid.mat
!tar -xf {DATA_ROOT}/102flowers.tgz -C {FLOWERS_DIR}

%cd /content/PromptSRC
DATA_ROOT = "/content/PromptSRC/data"
FLOWERS_DIR = os.path.join(DATA_ROOT, "oxford_flowers")
os.makedirs(FLOWERS_DIR, exist_ok=True)
json_path = os.path.join(FLOWERS_DIR, "cat_to_name.json")

if os.path.exists(json_path):
    os.remove(json_path)

!wget -q -O {json_path} https://raw.githubusercontent.com/udacity/aipnd-project/master/cat_to_name.json

with open(json_path, 'r') as f:
  data = json.load(f)

drive.mount('/content/drive')

SOURCE_ZIP = f"/content/drive/MyDrive/{MY_ZIP_NAME}"
SEED_DIR = "/content/PromptSRC/output/base2new/train_base/oxford_flowers/shots_16/PromptSRC/vit_b16_c2_ep5_batch4_2ctx/seed1"
os.makedirs(SEED_DIR, exist_ok=True)


!unzip -o -q "{SOURCE_ZIP}" -d "{SEED_DIR}"

/content
Cloning into 'PromptSRC'...
remote: Enumerating objects: 236, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 236 (delta 88), reused 199 (delta 67), pack-reused 0 (from 0)
Receiving objects: 100% (236/236), 32.99 MiB | 47.64 MiB/s, done.
Resolving deltas: 100% (88/88), done.
Cloning into 'Dassl.pytorch'...
remote: Enumerating objects: 2477, done.
remote: Counting objects: 100% (1081/1081), done.
remote: Compressing objects: 100% (235/235), done.
remote: Total 2477 (delta 933), reused 846 (delta 846), pack-reused 1396 (from 1)
Receiving objects: 100% (2477/2477), 410.19 KiB | 11.39 MiB/s, done.
Resolving deltas: 100% (1676/1676), done.
/content/Dassl.pytorch
/content/PromptSRC
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   348  100   348    0     0    401      0 --:--:-- --:--:-- --:--:--   40

In [2]:
config_dir = "configs/trainers/PromptSRC"
os.makedirs(config_dir, exist_ok=True)
config_file = os.path.join(config_dir, "vit_b16_c2_ep5_batch4_2ctx.yaml")
config_content = """
DATALOADER:
  TRAIN_X:
    BATCH_SIZE: 4
  TEST:
    BATCH_SIZE: 100
  NUM_WORKERS: 8
INPUT:
  SIZE: (224, 224)
  INTERPOLATION: "bicubic"
  PIXEL_MEAN: [0.48145466, 0.4578275, 0.40821073]
  PIXEL_STD: [0.26862954, 0.26130258, 0.27577711]
  TRANSFORMS: ["random_resized_crop", "random_flip", "normalize"]
OPTIM:
  NAME: "sgd"
  LR: 0.0035
  MAX_EPOCH: 50
  LR_SCHEDULER: "cosine"
  WARMUP_EPOCH: 1
  WARMUP_TYPE: "constant"
  WARMUP_CONS_LR: 1e-5
TRAIN:
  PRINT_FREQ: 20
MODEL:
  BACKBONE:
    NAME: "ViT-B/16"
TRAINER:
  PROMPTSRC:
    N_CTX: 2
    CTX_INIT: "a photo of a"
    PREC: "fp16"
    PROMPT_DEPTH_TEXT: 9
    PROMPT_DEPTH_VISION: 9
    TEXT_LOSS_WEIGHT: 25.0
    IMAGE_LOSS_WEIGHT: 10.0
    GPA_MEAN: 0.1
    GPA_STD: 0.1
"""
with open(config_file, "w") as f:
    f.write(config_content)

!sed -i 's/super().__init__(optimizer, last_epoch, verbose)/super().__init__(optimizer, last_epoch)/' /content/Dassl.pytorch/dassl/optim/lr_scheduler.py


In [3]:
new_train_code = """
import argparse
import torch
from dassl.utils import setup_logger, set_random_seed, collect_env_info
from dassl.config import get_cfg_default
from dassl.engine import build_trainer
from yacs.config import CfgNode as CN

# --- IMPORTS ---
from trainers import promptsrc
from datasets import oxford_flowers
# ---------------

def print_args(args, cfg):
    print("***************")
    print("** Arguments **")
    print("***************")
    optkeys = list(args.__dict__.keys())
    optkeys.sort()
    for key in optkeys:
        print("{}: {}".format(key, args.__dict__[key]))
    print("************")
    print("** Config **")
    print("************")
    print(cfg)

def reset_cfg(cfg, args):
    if args.root:
        cfg.DATASET.ROOT = args.root
    if args.output_dir:
        cfg.OUTPUT_DIR = args.output_dir
    if args.resume:
        cfg.RESUME = args.resume
    if args.seed:
        cfg.SEED = args.seed
    if args.source_domains:
        cfg.DATASET.SOURCE_DOMAINS = args.source_domains
    if args.target_domains:
        cfg.DATASET.TARGET_DOMAINS = args.target_domains
    if args.transforms:
        cfg.INPUT.TRANSFORMS = args.transforms
    if args.trainer:
        cfg.TRAINER.NAME = args.trainer
    if args.backbone:
        cfg.MODEL.BACKBONE.NAME = args.backbone
    if args.head:
        cfg.MODEL.HEAD.NAME = args.head

def extend_cfg(cfg):
    # Register PromptSRC Keys
    if not hasattr(cfg.TRAINER, "PROMPTSRC"):
            cfg.TRAINER.PROMPTSRC = CN()
            # --- UPDATED DIMENSIONS TO MATCH WEIGHTS (4) ---
            cfg.TRAINER.PROMPTSRC.N_CTX = 4        # Changed from 2 to 4
            cfg.TRAINER.PROMPTSRC.N_CTX_VISION = 4 # Changed from 0 to 4
            cfg.TRAINER.PROMPTSRC.N_CTX_TEXT = 4   # Changed from 0 to 4
            # -----------------------------------------------
            cfg.TRAINER.PROMPTSRC.CTX_INIT = ""
            cfg.TRAINER.PROMPTSRC.PREC = "fp16"
            cfg.TRAINER.PROMPTSRC.PROMPT_DEPTH_TEXT = 9
            cfg.TRAINER.PROMPTSRC.PROMPT_DEPTH_VISION = 9
            cfg.TRAINER.PROMPTSRC.TEXT_LOSS_WEIGHT = 25.0
            cfg.TRAINER.PROMPTSRC.IMAGE_LOSS_WEIGHT = 10.0
            cfg.TRAINER.PROMPTSRC.GPA_MEAN = 0.1
            cfg.TRAINER.PROMPTSRC.GPA_STD = 0.1

    # Register Subsample Key
    if not hasattr(cfg.DATASET, "SUBSAMPLE_CLASSES"):
        cfg.DATASET.SUBSAMPLE_CLASSES = "all"

def setup_cfg(args):
    cfg = get_cfg_default()
    extend_cfg(cfg)
    reset_cfg(cfg, args)
    if args.dataset_config_file:
        cfg.merge_from_file(args.dataset_config_file)
    if args.config_file:
        cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    cfg.freeze()
    return cfg

def main(args):
    cfg = setup_cfg(args)
    if cfg.SEED >= 0:
        print("Setting fixed seed: {}".format(cfg.SEED))
        set_random_seed(cfg.SEED)
    setup_logger(cfg.OUTPUT_DIR)
    if torch.cuda.is_available() and cfg.USE_CUDA:
        torch.backends.cudnn.benchmark = True
    print_args(args, cfg)
    print("Collecting env info ...")
    print("** System info **\\n{}".format(collect_env_info()))
    trainer = build_trainer(cfg)
    if args.eval_only:
        trainer.load_model(args.model_dir, epoch=args.load_epoch)
        trainer.test()
        return
    if not args.no_train:
        trainer.train()

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--root", type=str, default="", help="path to dataset")
    parser.add_argument("--output-dir", type=str, default="", help="output directory")
    parser.add_argument("--resume", type=str, default="", help="checkpoint directory")
    parser.add_argument("--seed", type=int, default=-1, help="only positive value enables a fixed seed")
    parser.add_argument("--source-domains", type=str, nargs="+", help="source domains for DA/DG")
    parser.add_argument("--target-domains", type=str, nargs="+", help="target domains for DA/DG")
    parser.add_argument("--transforms", type=str, nargs="+", help="data augmentation transforms")
    parser.add_argument("--config-file", type=str, default="", help="path to config file")
    parser.add_argument("--dataset-config-file", type=str, default="", help="path to config file for dataset")
    parser.add_argument("--trainer", type=str, default="", help="name of trainer")
    parser.add_argument("--backbone", type=str, default="", help="name of CNN backbone")
    parser.add_argument("--head", type=str, default="", help="name of head")
    parser.add_argument("--eval-only", action="store_true", help="evaluation only")
    parser.add_argument("--model-dir", type=str, default="", help="load model from this directory")
    parser.add_argument("--load-epoch", type=int, help="load model weights at this epoch")
    parser.add_argument("--no-train", action="store_true", help="do not train")
    parser.add_argument("opts", default=None, nargs=argparse.REMAINDER, help="modify config options")
    args = parser.parse_args()
    main(args)
"""

with open("train.py", "w") as f:
    f.write(new_train_code)

In [4]:
script_path = "scripts/base2new_test_promptsrc_flowers.sh"
script_content = f"""#!/bin/bash
DATASET=oxford_flowers
SEED=1
CFG=vit_b16_c2_ep5_batch4_2ctx
SHOTS=16
LOADEP=50
SUB=new
COMMON_DIR=${{DATASET}}/shots_${{SHOTS}}/PromptSRC/${{CFG}}/seed${{SEED}}
MODEL_DIR=output/base2new/train_base/${{COMMON_DIR}}
DIR=output/base2new/test_${{SUB}}/${{COMMON_DIR}}

python train.py \\
--root /content/PromptSRC/data \\
--seed ${{SEED}} \\
--trainer PromptSRC \\
--dataset-config-file configs/datasets/${{DATASET}}.yaml \\
--config-file configs/trainers/PromptSRC/${{CFG}}.yaml \\
--output-dir ${{DIR}} \\
--model-dir ${{MODEL_DIR}} \\
--load-epoch ${{LOADEP}} \\
--eval-only \\
DATASET.NUM_SHOTS ${{SHOTS}} \\
DATASET.SUBSAMPLE_CLASSES ${{SUB}} \\
DATALOADER.NUM_WORKERS 0
"""
with open(script_path, "w") as f:
    f.write(script_content)

In [5]:
!bash scripts/base2new_test_promptsrc_flowers.sh

2025-12-09 06:56:04.330942: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765263364.351529    1904 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765263364.357911    1904 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765263364.373194    1904 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765263364.373220    1904 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765263364.373224    1904 computation_placer.cc:177] computation placer alr

In [6]:
import os


%cd /content/PromptSRC
DATASET = "oxford_flowers"
SEED = 1
IMPROVED_DIR = "output/base2new/golden_config/oxford_flowers/shots_16/PromptSRC/seed1"


print(" Creating vit_b16_c2_ep50_flowers.yaml...")

golden_config_content = """
DATALOADER:
  TRAIN_X:
    BATCH_SIZE: 4
  TEST:
    BATCH_SIZE: 100
  NUM_WORKERS: 0

INPUT:
  SIZE: (224, 224)
  INTERPOLATION: "bicubic"
  PIXEL_MEAN: [0.48145466, 0.4578275, 0.40821073]
  PIXEL_STD: [0.26862954, 0.26130258, 0.27577711]
  TRANSFORMS: ["random_resized_crop", "random_flip", "normalize"]

OPTIM:
  NAME: "sgd"
  LR: 0.0025
  MAX_EPOCH: 50
  LR_SCHEDULER: "cosine"
  WARMUP_EPOCH: 1
  WARMUP_TYPE: "constant"
  WARMUP_CONS_LR: 1e-5

TRAIN:
  PRINT_FREQ: 20

MODEL:
  BACKBONE:
    NAME: "ViT-B/16"

TRAINER:
  PROMPTSRC:
    N_CTX_VISION: 4
    N_CTX_TEXT: 4
    CTX_INIT: "a photo of a"
    PREC: "fp16"
    PROMPT_DEPTH_VISION: 9
    PROMPT_DEPTH_TEXT: 9
    TEXT_LOSS_WEIGHT: 25.0
    IMAGE_LOSS_WEIGHT: 10.0
    GPA_MEAN: 45.0
    GPA_STD: 5.0
"""

config_path = "configs/trainers/PromptSRC/vit_b16_c2_ep50_flowers.yaml"
with open(config_path, "w") as f:
    f.write(golden_config_content)

!python train.py \
    --root /content/PromptSRC/data \
    --seed {SEED} \
    --trainer PromptSRC \
    --dataset-config-file configs/datasets/{DATASET}.yaml \
    --config-file {config_path} \
    --output-dir {IMPROVED_DIR} \
    DATASET.NUM_SHOTS 16 \
    DATASET.SUBSAMPLE_CLASSES base \
    DATALOADER.NUM_WORKERS 0

!python train.py \
    --root /content/PromptSRC/data \
    --seed {SEED} \
    --trainer PromptSRC \
    --dataset-config-file configs/datasets/{DATASET}.yaml \
    --config-file {config_path} \
    --output-dir {IMPROVED_DIR}/test_new \
    --model-dir {IMPROVED_DIR} \
    --load-epoch 50 \
    --eval-only \
    DATASET.NUM_SHOTS 16 \
    DATASET.SUBSAMPLE_CLASSES base \
    DATALOADER.NUM_WORKERS 0

/content/PromptSRC
 Creating vit_b16_c2_ep50_flowers.yaml...
2025-12-09 06:57:04.732588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765263424.806749    2215 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765263424.840161    2215 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765263424.914093    2215 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765263424.915566    2215 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765263424.915

In [7]:
import os


%cd /content/PromptSRC
DATASET = "oxford_flowers"
SEED = 1
IMPROVED_DIR = "output/base2new/golden_config/oxford_flowers/shots_16/PromptSRC/seed1"


print(" Creating vit_b16_c2_ep50_flowers.yaml...")

golden_config_content = """
DATALOADER:
  TRAIN_X:
    BATCH_SIZE: 4
  TEST:
    BATCH_SIZE: 100
  NUM_WORKERS: 0

INPUT:
  SIZE: (224, 224)
  INTERPOLATION: "bicubic"
  PIXEL_MEAN: [0.48145466, 0.4578275, 0.40821073]
  PIXEL_STD: [0.26862954, 0.26130258, 0.27577711]
  TRANSFORMS: ["random_resized_crop", "random_flip", "normalize"]

OPTIM:
  NAME: "sgd"
  LR: 0.0025
  MAX_EPOCH: 50
  LR_SCHEDULER: "cosine"
  WARMUP_EPOCH: 1
  WARMUP_TYPE: "constant"
  WARMUP_CONS_LR: 1e-5

TRAIN:
  PRINT_FREQ: 20

MODEL:
  BACKBONE:
    NAME: "ViT-B/16"

TRAINER:
  PROMPTSRC:
    N_CTX_VISION: 4
    N_CTX_TEXT: 4
    CTX_INIT: "a photo of a"
    PREC: "fp16"
    PROMPT_DEPTH_VISION: 9
    PROMPT_DEPTH_TEXT: 9
    TEXT_LOSS_WEIGHT: 25.0
    IMAGE_LOSS_WEIGHT: 10.0
    GPA_MEAN: 45.0
    GPA_STD: 5.0
"""

config_path = "configs/trainers/PromptSRC/vit_b16_c2_ep50_flowers.yaml"
with open(config_path, "w") as f:
    f.write(golden_config_content)

!python train.py \
    --root /content/PromptSRC/data \
    --seed {SEED} \
    --trainer PromptSRC \
    --dataset-config-file configs/datasets/{DATASET}.yaml \
    --config-file {config_path} \
    --output-dir {IMPROVED_DIR} \
    DATASET.NUM_SHOTS 16 \
    DATASET.SUBSAMPLE_CLASSES base \
    DATALOADER.NUM_WORKERS 0

!python train.py \
    --root /content/PromptSRC/data \
    --seed {SEED} \
    --trainer PromptSRC \
    --dataset-config-file configs/datasets/{DATASET}.yaml \
    --config-file {config_path} \
    --output-dir {IMPROVED_DIR}/test_new \
    --model-dir {IMPROVED_DIR} \
    --load-epoch 50 \
    --eval-only \
    DATASET.NUM_SHOTS 16 \
    DATASET.SUBSAMPLE_CLASSES new \
    DATALOADER.NUM_WORKERS 0

/content/PromptSRC
 Creating vit_b16_c2_ep50_flowers.yaml...
2025-12-09 07:23:13.438738: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765264993.545501    8922 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765264993.592230    8922 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765264993.668390    8922 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765264993.671549    8922 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765264993.671

# Hyper parameter improvements

In [8]:
import os


%cd /content/PromptSRC
DATASET = "oxford_flowers"
SEED = 1
IMPROVED_DIR = "output/flowers_improvement_run2"


print(" Creating vit_b16_c2_ep50_flowers.yaml...")

golden_config_content = """
DATALOADER:
  TRAIN_X:
    BATCH_SIZE: 4
  TEST:
    BATCH_SIZE: 100
  NUM_WORKERS: 0

INPUT:
  SIZE: (224, 224)
  INTERPOLATION: "bicubic"
  PIXEL_MEAN: [0.48145466, 0.4578275, 0.40821073]
  PIXEL_STD: [0.26862954, 0.26130258, 0.27577711]
  TRANSFORMS: ["random_resized_crop", "random_flip", "normalize"]

OPTIM:
  NAME: "sgd"
  LR: 0.0025
  MAX_EPOCH: 50
  LR_SCHEDULER: "cosine"
  WARMUP_EPOCH: 1
  WARMUP_TYPE: "constant"
  WARMUP_CONS_LR: 1e-5

TRAIN:
  PRINT_FREQ: 20

MODEL:
  BACKBONE:
    NAME: "ViT-B/16"

TRAINER:
  PROMPTSRC:
    N_CTX_VISION: 4
    N_CTX_TEXT: 4
    CTX_INIT: "a photo of a"
    PREC: "fp16"
    PROMPT_DEPTH_VISION: 9
    PROMPT_DEPTH_TEXT: 9
    TEXT_LOSS_WEIGHT: 25.0
    IMAGE_LOSS_WEIGHT: 10.0
    GPA_MEAN: 40.0
    GPA_STD: 10.0
"""

config_path = "configs/trainers/PromptSRC/vit_b16_c2_ep50_flowers.yaml"
with open(config_path, "w") as f:
    f.write(golden_config_content)

!python train.py \
    --root /content/PromptSRC/data \
    --seed {SEED} \
    --trainer PromptSRC \
    --dataset-config-file configs/datasets/{DATASET}.yaml \
    --config-file {config_path} \
    --output-dir {IMPROVED_DIR} \
    DATASET.NUM_SHOTS 16 \
    DATASET.SUBSAMPLE_CLASSES base \
    DATALOADER.NUM_WORKERS 0

!python train.py \
    --root /content/PromptSRC/data \
    --seed {SEED} \
    --trainer PromptSRC \
    --dataset-config-file configs/datasets/{DATASET}.yaml \
    --config-file {config_path} \
    --output-dir {IMPROVED_DIR}/test_new \
    --model-dir {IMPROVED_DIR} \
    --load-epoch 50 \
    --eval-only \
    DATASET.NUM_SHOTS 16 \
    DATASET.SUBSAMPLE_CLASSES base \
    DATALOADER.NUM_WORKERS 0

/content/PromptSRC
 Creating vit_b16_c2_ep50_flowers.yaml...
2025-12-09 07:24:45.601554: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765265085.707896    9412 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765265085.733769    9412 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765265085.770293    9412 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765265085.772539    9412 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765265085.772

In [9]:
import os


%cd /content/PromptSRC
DATASET = "oxford_flowers"
SEED = 1
IMPROVED_DIR = "output/flowers_improvement_run2"


print(" Creating vit_b16_c2_ep50_flowers.yaml...")

golden_config_content = """
DATALOADER:
  TRAIN_X:
    BATCH_SIZE: 4
  TEST:
    BATCH_SIZE: 100
  NUM_WORKERS: 0

INPUT:
  SIZE: (224, 224)
  INTERPOLATION: "bicubic"
  PIXEL_MEAN: [0.48145466, 0.4578275, 0.40821073]
  PIXEL_STD: [0.26862954, 0.26130258, 0.27577711]
  TRANSFORMS: ["random_resized_crop", "random_flip", "normalize"]

OPTIM:
  NAME: "sgd"
  LR: 0.0025
  MAX_EPOCH: 50
  LR_SCHEDULER: "cosine"
  WARMUP_EPOCH: 1
  WARMUP_TYPE: "constant"
  WARMUP_CONS_LR: 1e-5

TRAIN:
  PRINT_FREQ: 20

MODEL:
  BACKBONE:
    NAME: "ViT-B/16"

TRAINER:
  PROMPTSRC:
    N_CTX_VISION: 4
    N_CTX_TEXT: 4
    CTX_INIT: "a photo of a"
    PREC: "fp16"
    PROMPT_DEPTH_VISION: 9
    PROMPT_DEPTH_TEXT: 9
    TEXT_LOSS_WEIGHT: 25.0
    IMAGE_LOSS_WEIGHT: 10.0
    GPA_MEAN: 40.0
    GPA_STD: 10.0
"""

config_path = "configs/trainers/PromptSRC/vit_b16_c2_ep50_flowers.yaml"
with open(config_path, "w") as f:
    f.write(golden_config_content)

!python train.py \
    --root /content/PromptSRC/data \
    --seed {SEED} \
    --trainer PromptSRC \
    --dataset-config-file configs/datasets/{DATASET}.yaml \
    --config-file {config_path} \
    --output-dir {IMPROVED_DIR} \
    DATASET.NUM_SHOTS 16 \
    DATASET.SUBSAMPLE_CLASSES base \
    DATALOADER.NUM_WORKERS 0

!python train.py \
    --root /content/PromptSRC/data \
    --seed {SEED} \
    --trainer PromptSRC \
    --dataset-config-file configs/datasets/{DATASET}.yaml \
    --config-file {config_path} \
    --output-dir {IMPROVED_DIR}/test_new \
    --model-dir {IMPROVED_DIR} \
    --load-epoch 50 \
    --eval-only \
    DATASET.NUM_SHOTS 16 \
    DATASET.SUBSAMPLE_CLASSES new \
    DATALOADER.NUM_WORKERS 0

/content/PromptSRC
 Creating vit_b16_c2_ep50_flowers.yaml...
2025-12-09 07:51:01.048799: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765266661.120938   16125 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765266661.152488   16125 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765266661.242329   16125 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765266661.245548   16125 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765266661.245

In [11]:
!zip -r dassl_pytorch.zip /content/Dassl.pytorch
from google.colab import files
files.download('dassl_pytorch.zip')


updating: content/Dassl.pytorch/ (stored 0%)
updating: content/Dassl.pytorch/LICENSE (deflated 41%)
updating: content/Dassl.pytorch/.isort.cfg (deflated 30%)
updating: content/Dassl.pytorch/tools/ (stored 0%)
updating: content/Dassl.pytorch/tools/parse_test_res.py (deflated 66%)
updating: content/Dassl.pytorch/tools/replace_text.py (deflated 68%)
updating: content/Dassl.pytorch/tools/train.py (deflated 70%)
updating: content/Dassl.pytorch/DATASETS.md (deflated 69%)
updating: content/Dassl.pytorch/dassl/ (stored 0%)
updating: content/Dassl.pytorch/dassl/metrics/ (stored 0%)
updating: content/Dassl.pytorch/dassl/metrics/accuracy.py (deflated 52%)
updating: content/Dassl.pytorch/dassl/metrics/__init__.py (deflated 36%)
updating: content/Dassl.pytorch/dassl/metrics/distance.py (deflated 70%)
updating: content/Dassl.pytorch/dassl/metrics/__pycache__/ (stored 0%)
updating: content/Dassl.pytorch/dassl/metrics/__pycache__/__init__.cpython-312.pyc (deflated 23%)
updating: content/Dassl.pytorch/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Streaming output truncated to the last 5000 lines.
  adding: content/PromptSRC/data/oxford_flowers/jpg/image_06519.jpg (deflated 0%)
  adding: content/PromptSRC/data/oxford_flowers/jpg/image_07535.jpg (deflated 1%)
  adding: content/PromptSRC/data/oxford_flowers/jpg/image_00444.jpg (deflated 1%)
  adding: content/PromptSRC/data/oxford_flowers/jpg/image_05356.jpg (deflated 0%)
  adding: content/PromptSRC/data/oxford_flowers/jpg/image_02481.jpg (deflated 1%)
  adding: content/PromptSRC/data/oxford_flowers/jpg/image_05798.jpg (deflated 1%)
  adding: content/PromptSRC/data/oxford_flowers/jpg/image_06346.jpg (deflated 0%)
  adding: content/PromptSRC/data/oxford_flowers/jpg/image_06360.jpg (deflated 1%)
  adding: content/PromptSRC/data/oxford_flowers/jpg/image_07449.jpg (deflated 0%)
  adding: content/PromptSRC/data/oxford_flowers/jpg/image_04889.jpg (deflated 1%)
  adding: content/PromptSRC/data/oxford_flowers/jpg/image_06406.jpg (deflated 0%)
  adding: content/PromptSRC/data/oxford_flowers

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
!zip -r promptsrc.zip /content/PromptSRC
from google.colab import files
files.download('promptsrc.zip')

Streaming output truncated to the last 5000 lines.
updating: content/PromptSRC/data/oxford_flowers/jpg/image_06519.jpg (deflated 0%)
updating: content/PromptSRC/data/oxford_flowers/jpg/image_07535.jpg (deflated 1%)
updating: content/PromptSRC/data/oxford_flowers/jpg/image_00444.jpg (deflated 1%)
updating: content/PromptSRC/data/oxford_flowers/jpg/image_05356.jpg (deflated 0%)
updating: content/PromptSRC/data/oxford_flowers/jpg/image_02481.jpg (deflated 1%)
updating: content/PromptSRC/data/oxford_flowers/jpg/image_05798.jpg (deflated 1%)
updating: content/PromptSRC/data/oxford_flowers/jpg/image_06346.jpg (deflated 0%)
updating: content/PromptSRC/data/oxford_flowers/jpg/image_06360.jpg (deflated 1%)
updating: content/PromptSRC/data/oxford_flowers/jpg/image_07449.jpg (deflated 0%)
updating: content/PromptSRC/data/oxford_flowers/jpg/image_04889.jpg (deflated 1%)
updating: content/PromptSRC/data/oxford_flowers/jpg/image_06406.jpg (deflated 0%)
updating: content/PromptSRC/data/oxford_flowers

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>